## PROYECTO ETL

#### HONDA MOTORCYCLES

https://www.honda.es/motorcycles.html

In [22]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC  
warnings.filterwarnings('ignore')
options=Options()

In [23]:
options.add_argument('--start-maximized')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)


In [24]:
PATH = ChromeDriverManager().install()

In [25]:
driver = webdriver.Chrome(PATH, options=options)

In [26]:
url = 'https://www.honda.es/motorcycles.html'

In [27]:
driver.get(url)

In [28]:
# Acepto las cookies
cookies = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
cookies.click()
time.sleep(0.5)

In [29]:
# Accedo a las gamas de motos
gamas = driver.find_elements(By.XPATH, '//*[@id="outer"]/div[1]/header/div[2]/nav/ul/li[1]/a')
gamas[0].click()
time.sleep(2)

In [30]:
gama_names = driver.find_elements(By.XPATH, '//*[@id="page"]/section/div[2]/div[2]')

In [31]:
string = gama_names[0].text

In [32]:
gama_list = string.split('\n')

In [33]:
gama_list = [item.strip().lower().replace(" ", "-").split("(")[0].rstrip("-") for item in gama_list]
#gama_list

In [34]:
elems = driver.find_elements(By.XPATH, "//a[@href]")

In [35]:
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))

In [36]:
#Aquí me da todos los links de todas las gamas y los modelos que componen dichas gamas
links = [elem.get_attribute("href") for elem in elems]

In [37]:
# Si el link contiene 'specifications-and-price' guardarlo en una lista y así filtro segun lo que estoy buscando
links_specifications = []
for e in links:
    if 'specifications' in e:
        links_specifications.append(e)

In [38]:
#Bucle list comprehension
links_especifications = [e for e in links if 'specifications' in e]

In [39]:
final_links = [e for e in links_specifications if not 'off-road' in e]

In [40]:
driver.get(url)

In [41]:
columns_filtering = ['Tipo de motor',
 'Sistema frenado Trasero',
 'Diámetro x carrera (mm)',
 'Arranque',
 'Capacidad de combustible',
 'Peso en orden de marcha (kg)',
 'Panel de instrumentos',
 'Embrague',
 'Suspensión trasera',
 'Rueda trasera',
 'Sistema alimentación combustible',
 'Suspensión delantera',
 'Rueda delantera',
 'Transmisión final',
 'Potencia máxima',
 'Cilindrada (cm3)',
 'Relación de compresión',
 'Faro delantero',
 'Sistema frenado delantero',
 'Capacidad de la batería (VAh)',
 'Altura del asiento (mm)']

In [42]:
dfs = []
for link in final_links:
    gama = link.split('/')[5]
    name = link.split('/')[6]
    driver.get(link)
    try:
        price = driver.find_elements(By.XPATH, '//*[@id="page"]/div[1]/div[2]/div[1]/div/div/div[1]/div[2]/dl[1]/dd/span')[0].text
    except:
        price = driver.find_elements(By.XPATH, '//*[@id="page"]/div[1]/div[2]/div[2]/div/div/div[1]/div[2]/dl[1]/dd')[0].text
    
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    title = soup.find_all('td', class_="title")
    titles_clean = [e.text for e in title]
    results = soup.find_all('td', class_="result")
    results_clean = [e.text for e in results]
    list_of_tuples = list(zip(titles_clean, results_clean))
    df = pd.DataFrame.from_records(list_of_tuples)
    df_transpose = df.T
    new_header = df_transpose.iloc[0]
    df_final = df_transpose[1:] 
    df_final.columns = new_header
    df_final[columns_filtering]
    df_final["Gama"] = gama
    df_final["Nombre"] = name
    df_final["Precio"] = price
    dfs.append(df_final)
    


In [44]:
df_motos = pd.concat(dfs)

In [45]:
df_motos.head()

,Diámetro x carrera (mm),Sistema alimentación combustible,Relación de compresión,Cilindrada (cm3),Tipo de motor,Sistema de encendido,Potencia máxima,Par máximo,Arranque,Diámetro mariposa,...,Nombre,Precio,Sistema ABS,Señal de parada de emergencia,Quick Shifter,Luz diurna,Conectividad,Cancelación automática de intermitentes,Control de crucero,Luz de curva
1,"81mm x 48,5mm",Sistema de inyección programada de gasolina (P...,13:1,1000cc,"4 tiempos. 4 cilindros en línea, DOHC, refrige...",Digital transistorizado con control computeriz...,160kW (217CV),112Nm/12.500rpm,Eléctrico,52mm,...,cbr1000rr-r-fireblade-30th-anniversary,"30.600,00€*",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"81mm x 48,5mm",Sistema de inyección programada de gasolina (P...,"13,4:1",1.000cc,"4 tiempos. 4 cilindros en línea, DOHC, 16 válv...",Digital transistorizado con control computeriz...,160kW (217CV),NaN,Eléctrico,52mm,...,cbr1000rr-r-fireblade-sp,"30.100,00€*",2 canales con IMU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"81mm x 48,5mm",Sistema de inyección programada de gasolina (P...,"13,4:1",1000cc,"4 tiempos. 4 cilindros en línea, DOHC, 16 válv...",Digital transistorizado con control computeriz...,160kW (217CV),NaN,Eléctrico,52mm,...,cbr1000rr-r-fireblade,"25.450,00€*",2 canales con IMU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,67 x 46mm,Sistema de inyección programada de gasolina (P...,"11,6 : 1",649cc,"4 tiempos. 4 cilindros en línea, DOHC, 16 válv...",Totalmente transistorizado,70kW (95CV) @12.000rpm (95/1/EC),63Nm @9.500rpm (95/1/EC),Eléctrico,32mm,...,cbr650r,"9.700,00€*",Sistema de 2 canales,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"67 x 66,8mm",Sistema de inyección programada de gasolina (P...,"10,7 : 1",471cc,"4 tiempos, bicilíndrico paralelo, DOHC, 8 válv...",NaN,35kW (47CV) @8.600 rpm (95/1/EC),43Nm @6.500rpm (95/1/EC),Eléctrico,34mm,...,cbr500r,"7.250,00€*",De 2 canales,Sí,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df_motos.to_csv('honda.csv', header=True, index=False)